In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport ind.alpha191


In [9]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    
    df_ids = data.index.get_level_values(0)
    
    benchmark = smpl.get_benchmark('沪深300', start=df_ids.min(), end=df_ids.max()).data
    benchmark = benchmark[['open','close']].reset_index('code',drop=True)
    data['bm_index_open'] = df_ids.map(benchmark['open'])
    data['bm_index_close'] = df_ids.map(benchmark['close'])
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)


In [8]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',codes))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [10]:
df_treated = pretreate_data(df_all)

In [258]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
# ids = [92,157,149,191]

# ['000001','601728','000002','601825', '601868']
ids = np.arange(1,192)
np.random.shuffle(ids)

def generate_alpha_factors(fun_ids, df=None):
    import ind.alpha191 as a191
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np

    for i in fun_ids:
    # for i in ids:
        fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
        if fun_name != 'alpha030':
            indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
        else:
            indx = getattr(a191, fun_name)(df)
        # indx = excute_for_multidates(df_treated.loc[(slice(None),['000001','601728','000002','601825', '601868']),:], lambda x: getattr(a191, fun_name)(x) ,level=1)
        indx.name = '191_'+fun_name
        save_cache('{}_train'.format(indx.name), indx, cache_type=CACHE_TYPE.FACTOR)
        # print(indx)

# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


In [200]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'+ str(y.index[0])
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                Y_ =  Y[X.index]
                if Y_.isnull().any():
                    return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y_.values.reshape(-1, 1)).coef_[0]
            else:
                # if Y.isnull().any():
                #     return np.nan
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se


def alpha149(data, dependencies=['close', 'bm_index_close'], max_window=253):
    # REGBETA(FILTER(RET,BANCHMARK_INDEX_CLOSE<DELAY(BANCHMARK_INDEX_CLOSE,1)),
    # FILTER(BANCHMARK_INDEX_CLOSE/DELAY(BANCHMARK_INDEX_CLOSE,1)-1,BANCHMARK_INDEX_CLOSE<DELAY(BANCHMARK_INDEX_CLOSE,1)),252)
    if len(data)< max_window:
        return pd.Series(np.nan,index=data.index)
    bm = data['bm_index_close']
    bm = (bm.diff(1) < 0.0)
    part1 = data['close'].pct_change(periods=1)[bm]
    part2 = data['close'].rolling(252).mean().pct_change(periods=1)[bm]
    try:
        alpha = REGBETA(part1,part2,252)
    except Exception as e:
        print(e)
        return pd.Series(np.nan,index=data.index)
    return alpha

# print(alpha181(df_treated.loc[(slice(None),['000001']),:]).to_list())


In [35]:
# a = df_treated.loc[(slice(None),['000001']),:]
# excute_for_multidates(df_treated, lambda x: getattr(a191, 'alpha149')(x) ,level=1)
# x = excute_for_multidates(df_treated, lambda x: alpha149(x) ,level=1)
# x = excute_for_multidates(df_treated.loc[(slice(None),['000001','605499','601728','601825','601868']),:], lambda x: alpha149(x) ,level=1)
# x.sort_index()

# with pd.option_context('display.max_rows', None): 
#     print(x.sort_index())

def alpha030(data, dependencies=['close', 'PB', 'MktValue'], max_window=81):
    if not 'code' in df_treated.index.names:
        print('alpha030 needs stocks more than one')
        return None
    elif len(df_treated.index.get_level_values('code').unique())<=1:
        print('alpha030 needs stocks more than one')

    ## 不是单票的，以后再说
    # WMA((REGRESI(RET,MKT,SMB,HML,60))^2,20)
    # 即特质性收益
    # MKT 为市值加权的市场平均收益率，
    # SMB 为市值最小的30%的股票的平均收益减去市值最大的30%的股票的平均收益，
    # HML 为PB最高的30%的股票的平均收益减去PB最低的30%的股票的平均收益    ret = data['close'].pct_change(periods=1).fillna(0.0)
    
    def alpha_30_deal(d_):
        print(d_.index)
        return d_
        mkt_ret = (ret * d_['cap']).sum(axis=1) / d_['cap'].sum(axis=1)
        me30 = (d_['cap'].T <= d_['cap'].quantile(0.3, axis=1)).T
        me70 = (d_['cap'].T >= d_['cap'].quantile(0.7, axis=1)).T
        pb30 = (d_['pb'].T <= d_['pb'].quantile(0.3, axis=1)).T
        pb70 = (d_['pb'].T >= d_['pb'].quantile(0.7, axis=1)).T
        smb_ret = ret[me30].mean(axis=1, skipna=True) - ret[me70].mean(axis=1, skipna=True)
        hml_ret = ret[pb70].mean(axis=1, skipna=True) - ret[pb30].mean(axis=1, skipna=True)
        xs = pd.concat([mkt_ret, smb_ret, hml_ret], axis=1)
        idxs = pd.Series(data=range(len(d_['close'].index)), index=d_['close'].index)

        def multi_var_linregress(idx, y, xs):
            X = xs.iloc[idx]
            Y = y.iloc[idx]
            X = sm.add_constant(X)
            try:
                res = np.array(sm.OLS(Y, X).fit().resid)
            except Exception as e:
                return pd.Series(np.nan,index=data.index)
            return res[-1]

        # print(xs.tail(5), ret.tail(5))
        residual = [idxs.rolling(window=60, min_periods=60).apply(lambda x: multi_var_linregress(x, ret[col], xs)) for col in ret.columns]
        residual = pd.concat(residual, axis=1)
        residual.columns = ret.columns

        # w = preprocessing.normalize(np.array([i for i in range(1, 21)]), norm='l1', axis=1).reshape(-1)
        # w = preprocessing.normalize(np.array([i for i in range(1, 21)]).reshape(-1, 1), norm='l1', axis=0).reshape(-1)
        w = np.array(range(1, 21))
        w = w/w.sum()
        alpha = (residual ** 2).rolling(window=20, min_periods=20).apply(lambda x: np.dot(x, w))
        return alpha
    

    return excute_for_multidates(data, lambda x: alpha_30_deal(x),level=0)

# alpha030(df_treated.loc[(slice(None),['000001','000002']),:])

# def alpha186(data, dependencies=['ADXR'], max_window=1):
#     return None
#     # \u5c31\u662fADXR
# #     (MEAN(ABS(SUM((LD>0  &  LD>HD)?LD:0,14)*100/SUM(TR,14)-SUM((HD>0  &
# # HD>LD)?HD:0,14)*100/SUM(TR,14))/(SUM((LD>0  &  LD>HD)?LD:0,14)*100/SUM(TR,14)+SUM((HD>0  &
# # HD>LD)?HD:0,14)*100/SUM(TR,14))*100,6)+DELAY(MEAN(ABS(SUM((LD>0  &
# # LD>HD)?LD:0,14)*100/SUM(TR,14)-SUM((HD>0  &  HD>LD)?HD:0,14)*100/SUM(TR,14))/(SUM((LD>0  &
# # LD>HD)?LD:0,14)*100/SUM(TR,14)+SUM((HD>0 & HD>LD)?HD:0,14)*100/SUM(TR,14))*100,6),6))/2
#     return data['ADXR'].iloc[-1]

# excute_for_multidates(df_treated.loc[(slice(None),['000001','000002']),:], lambda x: a191.alpha021(x) ,level=1)


In [102]:
# df_treated.index.get_level_values(1).unique().to_list()
# '601728',
#  '601825',
#  '601868']
# alpha149(df_treated.loc[(slice(None),['601825']),:])


# a = df_treated.loc[(pd.Timestamp('2017-01-05'),['000001','000002']),:]
a = df_treated.loc[(pd.Timestamp('2017-01-05'),['000001', '000002', '000063', '000066', '000069', '000100', '000157','000166', '000301', '000333']),:]

def test(d):
    # ret = d['returns']
    # mkt_ret = (ret * d['cap']).sum() / d['cap'].sum()
    # return mkt_ret
    me30 = (d['cap'] <= d['cap'].quantile(0.3))
    me70 = (d['cap'] >= d['cap'].quantile(0.7))
    pb30 = (d['pb'] <= d['pb'].quantile(0.3))
    pb70 = (d['pb'] >= d['pb'].quantile(0.7))
    
    
excute_for_multidates(a, lambda x: test(x),level=0)
# df_treated.loc[(pd.Timestamp('2017-01-05'),['000001','000002']),:].sort_index()[['returns','cap']]

""


In [5]:
%%time
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]
# pd.Series(-1,index=df_treated.loc[(slice(None),['000001']),:].index,dtype=np.dtype('int8'))

# a=alpha004(df_treated.loc[(slice(None),['000001']),:])
# a.rolling(window=5, min_periods=5).corr(a).to_list()
# a.to_list()
# a.rolling(window=5, min_periods=5).apply(lambda x:x.corr(a[x.index])).to_list()
# a[0:50].rolling(window=5, min_periods=5).apply(lambda x:print(x.corr(a[x.index])))



Wall time: 0 ns
